In [7]:
!pip install numpy
!pip install boto3
!pip install torch
import numpy as np
import utils

import torch
import torch.nn as nn
import torch.optim as optim

import boto3
import pickle

In [8]:
import os

BASE_IMAGE = 'docker.io/pytorch/pytorch:1.0-cuda10.0-cudnn7-runtime'
bucket_name = 'elyra'
S3_END_POINT = "http://minio-kubeflow.apps.openmaru.ocp482.com"
S3_ACCESS_ID = "minio"
S3_ACCESS_KEY = "minio123"

In [9]:
s3_endpoint_url = 'http://minio-kubeflow.apps.openmaru.ocp482.com'
s3_access_key = "minio"
s3_secret_key = "minio123"

s3 = boto3.client(service_name='s3',
              	aws_access_key_id = s3_access_key,
              	aws_secret_access_key = s3_secret_key,
              	endpoint_url=s3_endpoint_url)

s3.list_buckets()['Buckets']

[{'Name': 'elyra',
  'CreationDate': datetime.datetime(2021, 8, 15, 12, 40, 14, 518000, tzinfo=tzlocal())},
 {'Name': 'mlpipeline',
  'CreationDate': datetime.datetime(2021, 7, 29, 13, 47, 55, 26000, tzinfo=tzlocal())}]

In [10]:
class Net(nn.Module):
    def __init__(self, n_inputs=2, n_outputs=1, n_hidden_nodes=10, n_hidden_layers=1, activation=nn.ReLU(), output_activation=None):
        super(Net, self).__init__()

        self.layer_list = nn.ModuleList()

        for i in range(n_hidden_layers):
            if i==0:
                self.layer_list.append(nn.Linear(n_inputs, n_hidden_nodes))
            else:
                self.layer_list.append(nn.Linear(n_hidden_nodes, n_hidden_nodes))
        
        self.output_layer = nn.Linear(n_hidden_nodes, n_outputs)

        self.activation = activation
        self.output_activation = output_activation

    def forward(self, x):
        out = x

        for layer in self.layer_list:
            out = self.activation(layer(out))

        out = self.output_layer(out)
        if self.output_activation is not None:
            out = self.output_activation(out)

        return out

In [11]:
def write_to_store(bucket, data, key):
    s3.put_object(Body=pickle.dumps(data),
                      Bucket=bucket,
                      Key=key)
    
def read_from_store(bucket, key):
    raw_data = s3.get_object(Bucket=bucket,
                                 Key=key)['Body']._raw_stream.data

    return pickle.loads(raw_data)

def train_model(hyperparam_idx: int, retcode_download: int, N_gridsize: int) -> int:
    '''Look up hyperparams from store
    and train model
    '''

    if hyperparam_idx >= N_gridsize:
        raise ValueError("hyperparam_idx cannot be >= N_gridsize")

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f'Device = {device}')


    features_train = torch.from_numpy(read_from_store(bucket_name, 'features_train')).float()
    target_train = torch.from_numpy(read_from_store(bucket_name, 'target_train')).float()
    features_test = torch.from_numpy(read_from_store(bucket_name, 'features_test')).float()
    target_test = torch.from_numpy(read_from_store(bucket_name, 'target_test')).float()

    conf = read_from_store(bucket_name, 'hyperparam_grid')[hyperparam_idx]
    lr = float(conf.get('lr', 1e-2))
    N_epochs = int(conf.get('N_epochs', 1000))
    num_hidden_layers = int(conf.get('num_hidden_layers', 1))
    num_nodes = int(conf.get('num_nodes', 2))
    activation = conf.get('activation', 'relu')

    #should be dependent on vars read from config
    if activation=='relu':
        activation = nn.ReLU()
    elif activation=='sigmoid':
        activation = nn.Sigmoid()

    model = Net(n_inputs=2, n_outputs=1, n_hidden_nodes=num_nodes, n_hidden_layers=num_hidden_layers, activation=activation, output_activation=nn.Sigmoid())
    #model = nn.Sequential(nn.Linear(2, 10), nn.ReLU(), nn.Linear(10, 1), nn.Sigmoid())
    
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Adam optimizer
    model.train()    

    if device!='cpu':
        model = model.to(device)
        features_train = features_train.to(device)
        target_train = target_train.to(device)

    for epoch in range(N_epochs): #N_epochs = number of iterations over the full dataset
        features_shuffled = features_train
        target_shuffled = target_train

        out = model(features_shuffled) #predictions from model
        loss = criterion(out.squeeze(), target_shuffled.squeeze()) #loss between predictions and labels

        if epoch % 1000 == 0:
            print(f'epoch = {epoch} loss = {loss}')

        optimizer.zero_grad()
        loss.backward() #compute gradients
        optimizer.step() #update model

    out = model(features_shuffled) #predictions from model
    train_loss = criterion(out.squeeze(), target_shuffled.squeeze()) #loss between predictions and labels
    print(f'Train Loss : {train_loss}')

    def evaluate_model(model, features_test, target_test):
        '''Evaluate model on test set
        and store result
        '''
        model.eval()

        if device!='cpu':
            features_test = features_test.to(device)
            target_test = target_test.to(device)

        out = model(features_test)
        loss = criterion(out.squeeze(), target_test.squeeze())
        

        return loss

    test_loss = evaluate_model(model, features_test, target_test)
    print(f'Test  Loss : {test_loss}')

    #write_to_store(bucket_name, {'test_loss': test_loss.item(), 'model': model}, f'score_{hyperparam_idx}', client)
    write_to_store(bucket_name, {'test_loss': test_loss.item()}, f'score_{hyperparam_idx}')

    return hyperparam_idx


In [12]:
retcode_download = 0
N_gridsize = 18
hyperparam_idx = 0 #this should be a notebook parameter

retcode = train_model(hyperparam_idx, retcode_download, N_gridsize)

Device = cpu
epoch = 0 loss = 0.7473655343055725
Train Loss : 0.6931474208831787
Test  Loss : 0.6931473612785339
